# Preprocess Dataset

reads data from `train_dir` and outputs processed images to `processed_train_dir`. <br>
processing => 
1. convert to grayscale depending on the flag
2. resize to 128, 128 pixels


In [26]:
# imports
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import sys

In [3]:
train_dir = 'dataset/train'
labels_csv = 'dataset/labels.csv'
processed_train_dir = 'dataset/processed/hub/train'

size = 299, 299
n_files = len(os.listdir(train_dir))

labels_df = pd.read_csv(labels_csv)

In [32]:
def load_image(filename):
    img = Image.open(os.path.join(train_dir, filename))
    return np.asarray(img, dtype="int32").flatten() / 255.0

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [24]:
g = labels_df['breed'].value_counts()
classes = list(g.keys())
n_classes = len(classes)

In [31]:
### FOR TRAIN DATA #####

train_filename = 'dataset/train.tfrecords'  # filepath to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)

for file_i, file_ in enumerate(os.listdir(train_dir)):
    
    
    img = load_image(file_)
    label = labels_df[labels_df['id'] == file_.strip('.jpg')]['breed'].values[0]
    # convert label to int
    label = classes.index(label)
    
    feature = {'train/label': _int64_feature(label),
               'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring()))}
    # Create an example protocol buffer
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    
    # Serialize to string and write on the file
    writer.write(example.SerializeToString())
    if file_i % 10 == 0:
        print "processed file {} out of {} \r".format(file_i + 1, n_files),
    
writer.close()
sys.stdout.flush()

AttributeError: 'module' object has no attribute 'Int32List'